In [ ]:
from hurst_exponent import *
# Upload csv file
upload_2 = files.upload()

Saving ushec.csv to ushec.csv


# **1. Hurst Exponent**
The Hurst Exponent (\(H\)) measures the long-term memory or persistence of a time series. It helps classify the series into one of three categories:
- **\(H < 0.5\):** Mean-reverting behavior (anti-persistent).
- **\(H = 0.5\):** Geometric Brownian motion (random walk, no memory).
- **\(H > 0.5\):** Trending behavior (persistent).

## **Why Perform the Hurst Exponent Test?**
- **Identify Trends or Mean Reversion:** Helps determine whether a time series is trending or mean-reverting.
- **Trading Strategies:** Traders use this information to design strategies:
  - **Trend-following strategies** for \(H > 0.5\).
  - **Mean-reversion strategies** for \(H < 0.5\).

---

In [ ]:
# Example usage for securities in Nepse
if __name__ == "__main__":
    results, data = analyze_securities('ushec.csv')
    symbol = data['Symbol'].iloc[0]
    print(f'An analysis for {symbol}:')
results

An analysis for USHEC:


,Hurst Exponent,Classification
0,1.01489,Trending


# **2. ADF Test (Augmented Dickey-Fuller Test)**
The ADF test checks for the presence of a **unit root** in a time series. It is used to determine whether a series is **stationary** or **non-stationary**.

- **Null Hypothesis \(H_0\):** The series has a unit root (non-stationary).
- **Alternative Hypothesis \(H_1\):** The series does not have a unit root (stationary).

## **Why Perform the ADF Test?**
- **Stationarity Check:** Many statistical models (e.g., ARIMA, regression) assume stationarity. Non-stationary data can lead to spurious results.
- **Differencing Requirement:** If the series is non-stationary, differencing is required to make it stationary before modeling.
- **Forecasting:** Stationary series are easier to model and forecast.

---

# **3. KPSS Test (Kwiatkowski-Phillips-Schmidt-Shin Test)**
The KPSS test is another test for stationarity, but it works differently from the ADF test. It tests the null hypothesis that the series is **stationary** around a deterministic trend.

- **Null Hypothesis \(H_0\):** The series is stationary.
- **Alternative Hypothesis \(H_1\):** The series is non-stationary.

## **Why Perform the KPSS Test?**
- **Complementary to ADF:** The KPSS test is often used alongside the ADF test to confirm stationarity. If the ADF test fails to reject the null (non-stationary) and the KPSS test rejects the null (non-stationary), it provides stronger evidence of non-stationarity.
- **Robustness:** The KPSS test is robust to certain types of structural breaks and can detect stationarity around a trend.

## **Intepretation**
- When interpreting the p-value, a small p-value (below your chosen significance level) indicates that you should reject the null hypothesis.

---

In [ ]:
# Apply the tests to data
kpss_results_df = kpss_test(data['Close'])
adf_results_df = adf_test(data['Close'])
print(f'KPSS for {symbol}')
kpss_results_df

KPSS for USHEC


,Test Statistic,p-value,Lags Used,Critical Value (10%),Critical Value (5%),Critical Value (2.5%),Critical Value (1%)
0,2.352399,0.01,15,0.347,0.463,0.574,0.739


In [ ]:
print(f'ADF for {symbol}')
adf_results_df

ADF for USHEC


,Test Statistic,p-value,Lags Used,Critical Value (1%),Critical Value (5%),Critical Value (10%)
0,0.388653,0.981066,19,-3.442495,-2.866897,-2.569623




## **Why Perform All Three Tests Together?**
1. **Comprehensive Analysis:**
   - The **Hurst Exponent** provides insights into the long-term behavior of the series (trending or mean-reverting).
   - The **ADF test** and **KPSS test** together provide a robust check for stationarity, which is critical for modeling and forecasting.

2. **Conflicting Results:**
   - Sometimes, the ADF and KPSS tests can give conflicting results. For example:
    - **Case 1:** KPSS stationary, ADF non-stationary → Series is **trend stationary**; detrend and retest.  
    - **Case 2:** KPSS non-stationary, ADF stationary → Series is **difference stationary**; difference and retest.  

3. **Model Selection:**
   - If the series is non-stationary, differencing or detrending may be required.
   - If the series is trending \(H > 0.5\), models like ARIMA with differencing or trend components may be appropriate.
   - If the series is mean-reverting \(H < 0.5\), models like mean-reversion strategies or Ornstein-Uhlenbeck processes may be suitable.

---

## **Summary Table**

| Test                | Purpose                                                                 | Key Insight                                                                 |
|---------------------|-------------------------------------------------------------------------|-----------------------------------------------------------------------------|
| **Hurst Exponent**  | Measure long-term memory or persistence                                 | Classifies series as trending, mean-reverting, or random walk.             |
| **ADF Test**        | Check for unit root (non-stationarity)                                  | Determines if differencing is required for stationarity.                   |
| **KPSS Test**       | Check for stationarity around a deterministic trend                     | Confirms stationarity or non-stationarity, complementary to ADF.           |



In [ ]:
# Interpret ADF results
adf_stationary = adf_results_df.iloc[:,1].values < adf_results_df.iloc[:,4].values  # ADF test statistic should be smaller than 5% critical value

# Interpret KPSS results
kpss_stationary = kpss_results_df.iloc[:,1].values < kpss_results_df.iloc[:,4].values  # KPSS test statistic should be smaller than 5% critical value

# Conclusion based on the results
if not adf_stationary and not kpss_stationary:
    conclusion = "The series is not stationary."
elif adf_stationary and kpss_stationary:
    conclusion = "The series is stationary."
elif not adf_stationary and kpss_stationary:
    conclusion = "The series is trend stationary. Detrending is needed."
elif adf_stationary and not kpss_stationary:
    conclusion = "The series is difference stationary. Differencing is needed."

print(conclusion)

The series is trend stationary. Detrending is needed.


# 2. Multiple tickers

In [ ]:
# Example usage
if __name__ == "__main__":
    tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "NFLX"]
    start_date = "2020-01-01"
    end_date = "2023-01-01"
    results_df = analyze_multiple_tickers(tickers, start_date, end_date)
    print("\nResults:")
    pd.set_option('display.max_colwidth', None)
results_df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Results:


,Ticker,Hurst Exponent,Classification,KPSS Test Statistic,KPSS p-value,KPSS Critical Value (5%),ADF Test Statistic,ADF p-value,ADF Critical Value (5%),Conclusion
0,AAPL,1.026769,Trending,3.356468,0.01,0.463,-1.888907,0.337297,-2.865381,The series is trend stationary. Detrending is needed.
1,MSFT,1.037112,Trending,2.786358,0.01,0.463,-1.858999,0.351588,-2.865422,The series is trend stationary. Detrending is needed.
2,GOOGL,1.039019,Trending,2.477872,0.01,0.463,-1.394341,0.584986,-2.865381,The series is trend stationary. Detrending is needed.
3,AMZN,0.995538,Trending,0.918475,0.01,0.463,-1.394058,0.585121,-2.865376,The series is trend stationary. Detrending is needed.
4,TSLA,1.035357,Trending,2.715034,0.01,0.463,-1.877528,0.342705,-2.865422,The series is trend stationary. Detrending is needed.
5,NFLX,1.031325,Trending,1.449053,0.01,0.463,-1.220341,0.664785,-2.865376,The series is trend stationary. Detrending is needed.
